# filter based on total tenure

In [2]:
import pandas as pd
import os
absolutepath = os.path.abspath(os.getcwd())
fileDirectory = os.path.dirname(absolutepath)
parentDirectory = os.path.dirname(fileDirectory)
dataprocDirectory = os.path.join(fileDirectory, 'data/proc')
datarawDirectory = os.path.join(fileDirectory, 'data/raw')

In [3]:
spells = pd.read_csv(os.path.join(dataprocDirectory, 'author_pool_unrestricted.csv'))
spells.shape
all_authors_in_dataset = spells[['authid']].drop_duplicates()
all_authors_in_dataset['in_dataset'] = 1
all_authors_in_dataset.to_csv(os.path.join(dataprocDirectory, 'author_pool_indicator.csv'),index=False)
os.path.join(dataprocDirectory, 'author_pool_indicator.csv')

'c:\\Users\\lixiang\\.ssh\\10282022_iteration_peer_effect_stacking\\data/proc\\author_pool_indicator.csv'

# create an indicator for being in the database
Drop table if exists all_authors_in_dataset_jttp;

CREATE TABLE all_authors_in_dataset_jttp (
  authid bigint,
  in_dataset int,
  PRIMARY KEY (authid)
);

\COPY all_authors_in_dataset_jttp FROM '/cluster/work/lawecon/Work/dcai/jiycai/thousand_talent/thousand_talent/scripts/10282022_iteration_peer_effect_stacking/data/proc/author_pool_indicator.csv' DELIMITER ',' CSV HEADER;

# run the following in the database

\copy (select * from scopus_authors_year_outcome_chinese_deduped 
       left join all_authors_in_dataset_jttp 
       on scopus_authors_year_outcome_chinese_deduped.authid = all_authors_in_dataset_jttp.authid 
       where all_authors_in_dataset_jttp.in_dataset = 1) 
to ''c:\\Users\\lixiang\\.ssh\\10282022_iteration_peer_effect_stacking\\data/proc\\author_pool_outcomes.csv' 
csv header;


# apply restriction

In [4]:

outcomes = pd.read_csv(os.path.join(dataprocDirectory, 'author_pool_outcomes.csv'))

# at least 3 years
# at least 5 papers

In [5]:
outcomes_authid_gb = outcomes[['authid','year','num_docs']].groupby(['authid']).agg({'year':['min','max'],
                                                                                     'num_docs':['sum']}).reset_index()

In [6]:
outcomes_authid_gb.columns = ['_'.join(col) for col in outcomes_authid_gb.columns]

In [7]:
outcomes_authid_gb['total_year'] = outcomes_authid_gb.year_max - outcomes_authid_gb.year_min + 1 

In [8]:
outcomes_authid_gb['restriction'] = 1*((outcomes_authid_gb.total_year>2)&
                                     (outcomes_authid_gb.num_docs_sum>4))

In [9]:
outcomes_authid_gb[outcomes_authid_gb.restriction==1].shape

(594481, 6)

In [10]:
outcomes_authid_gb.shape

(1800227, 6)

In [11]:
594481/1800227

0.33022557710777584

In [12]:
outcomes_authid_gb = outcomes_authid_gb[['authid_','restriction']]
outcomes_authid_gb.columns = ['authid','restriction']
outcomes_authid_gb.to_csv(os.path.join(dataprocDirectory, 'author_pool_restriction.csv'),index=False)

# create an indicator for being in within restriction
Drop table if exists all_authors_in_restriction_jttp;

CREATE TABLE all_authors_in_restriction_jttp (
  authid bigint,
  restriction int,
  PRIMARY KEY (authid)
);

\COPY all_authors_in_restriction_jttp FROM '/cluster/work/lawecon/Work/dcai/jiycai/thousand_talent/thousand_talent/scripts/10282022_iteration_peer_effect_stacking/data/proc/author_pool_restriction.csv' DELIMITER ',' CSV HEADER;

# create produce author_pool.csv

In [13]:
author_pool_unrestricted = pd.read_csv(os.path.join(dataprocDirectory, 'author_pool_unrestricted.csv'))

In [14]:
author_pool_unrestricted = author_pool_unrestricted.merge(outcomes_authid_gb,
                                                          how = 'left',
                                                          on = 'authid')

In [15]:
author_pool_unrestricted = author_pool_unrestricted[author_pool_unrestricted.restriction==1]

In [16]:
author_pool_unrestricted.to_csv(os.path.join(dataprocDirectory, 'author_pool.csv'),index=False)